## Basic Test

In [17]:
import cv2

video_path = "WhatsApp Video 2025-08-30 at 11.02.13 AM.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
else:
    print("Video opened successfully.")
    # You can add code here to process the video frames
    # cap.release() # Don't release yet if you plan to process frames later

Video opened successfully.


In [18]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Here we will add code for shuttlecock detection and tracking

    # For now, just display the frame
    # You might want to resize or process the frame before displaying
    cv2.imshow('Frame', frame)

    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the video capture object
cap.release()

# Close all the frames
cv2.destroyAllWindows()

## Main Code

In [3]:
# Cell 0 — Columnar slider UI (2 control columns + 1 preview window)
import cv2, numpy as np, json, time
from pathlib import Path

# --- CONFIG ---
video_path = "WhatsApp Video 2025-08-30 at 11.02.13 AM.mp4"
OUT_DIR = Path("outputs"); OUT_DIR.mkdir(parents=True, exist_ok=True)
SAVE_JSON = OUT_DIR / "mask_params.json"

DOWNSCALE = 1.0
DISPLAY_MAX_W = 1400

# --- CAPTURE ---
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise RuntimeError("Could not open video (check path).")

# --- WINDOWS (place 2 narrow columns on top, big preview below) ---
WIN_PREV = "Tuner"
WIN_C1   = "Controls: Color"
WIN_C2   = "Controls: Levels"

cv2.namedWindow(WIN_PREV, cv2.WINDOW_NORMAL)
cv2.namedWindow(WIN_C1,   cv2.WINDOW_NORMAL)
cv2.namedWindow(WIN_C2,   cv2.WINDOW_NORMAL)

# optional: position windows so they look like columns; tweak to your screen
try:
    cv2.moveWindow(WIN_C1, 20, 20)
    cv2.resizeWindow(WIN_C1, 320, 480)
    cv2.moveWindow(WIN_C2, 360, 20)
    cv2.resizeWindow(WIN_C2, 320, 480)
    cv2.moveWindow(WIN_PREV, 20, 520)
    cv2.resizeWindow(WIN_PREV, DISPLAY_MAX_W, 420)
except Exception:
    pass

# --- Trackbars in 2 columns ---
# Column 1: color space + HSV or RGB ranges
cv2.createTrackbar("ColorSpace 0=HSV 1=RGB", WIN_C1, 0, 1, lambda v: None)

# HSV (defaults for yellow/green)
for name, init, maxv in [
    ("H low", 30, 179), ("H high", 60, 179),
    ("S low", 120, 255), ("S high", 255, 255),
    ("V low", 150, 255), ("V high", 255, 255),
]:
    cv2.createTrackbar(name, WIN_C1, init, maxv, lambda v: None)

# RGB thresholds
for name, init in [
    ("R low", 0), ("R high", 255),
    ("G low", 180), ("G high", 255),
    ("B low", 0), ("B high", 120)
]:
    cv2.createTrackbar(name, WIN_C1, init, 255, lambda v: None)

# Column 2: contrast/brightness + motion fuse
cv2.createTrackbar("Alpha x100 (contrast)", WIN_C2, 100, 400, lambda v: None)  # 1.00..4.00
cv2.createTrackbar("Beta (-100..100)+100", WIN_C2, 100, 200, lambda v: None)   # 0..200 -> -100..100
cv2.createTrackbar("Fuse Motion (0/1)",     WIN_C2, 1,   1,   lambda v: None)

bg = cv2.createBackgroundSubtractorMOG2(history=600, varThreshold=32, detectShadows=False)

def tb(name, win): 
    return cv2.getTrackbarPos(name, win)

def build_mask(frame_bgr):
    # read from both columns
    cs    = tb("ColorSpace 0=HSV 1=RGB", WIN_C1)
    alpha = tb("Alpha x100 (contrast)", WIN_C2) / 100.0
    beta  = tb("Beta (-100..100)+100",  WIN_C2) - 100
    fuse  = tb("Fuse Motion (0/1)",     WIN_C2)

    adj = cv2.convertScaleAbs(frame_bgr, alpha=alpha, beta=beta)

    if cs == 0:  # HSV
        hL,hH = tb("H low", WIN_C1),  tb("H high", WIN_C1)
        sL,sH = tb("S low", WIN_C1),  tb("S high", WIN_C1)
        vL,vH = tb("V low", WIN_C1),  tb("V high", WIN_C1)
        hsv = cv2.cvtColor(adj, cv2.COLOR_BGR2HSV)
        if hL <= hH:
            lower = np.array([hL, sL, vL], np.uint8)
            upper = np.array([hH, sH, vH], np.uint8)
            mask_color = cv2.inRange(hsv, lower, upper)
        else:  # wrap-around
            l1,u1 = np.array([0,  sL, vL], np.uint8), np.array([hH, sH, vH], np.uint8)
            l2,u2 = np.array([hL, sL, vL], np.uint8), np.array([179,sH, vH], np.uint8)
            mask_color = cv2.bitwise_or(cv2.inRange(hsv, l1, u1), cv2.inRange(hsv, l2, u2))
    else:       # RGB
        rgb = cv2.cvtColor(adj, cv2.COLOR_BGR2RGB)
        rL,rH = tb("R low", WIN_C1), tb("R high", WIN_C1)
        gL,gH = tb("G low", WIN_C1), tb("G high", WIN_C1)
        bL,bH = tb("B low", WIN_C1), tb("B high", WIN_C1)
        lower = np.array([rL, gL, bL], np.uint8)
        upper = np.array([rH, gH, bH], np.uint8)
        mask_color = cv2.inRange(rgb, lower, upper)

    # light cleanup
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    mask_color = cv2.morphologyEx(mask_color, cv2.MORPH_OPEN, k, iterations=1)
    mask_color = cv2.morphologyEx(mask_color, cv2.MORPH_CLOSE, k, iterations=1)

    if fuse == 1:
        fg = bg.apply(adj)
        fg = cv2.medianBlur(fg, 5)
        mask = cv2.bitwise_and(mask_color, fg)
    else:
        mask = mask_color
    return adj, mask

def make_mosaic(orig_bgr, mask):
    mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    masked   = cv2.bitwise_and(orig_bgr, orig_bgr, mask=mask)
    row = np.hstack([orig_bgr, mask_bgr, masked])
    h,w = row.shape[:2]
    if w > DISPLAY_MAX_W:
        scale = DISPLAY_MAX_W / w
        row = cv2.resize(row, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    return row

print("Controls:  [S] Save JSON   [Space] Pause/Play   [Q] Quit")
paused, last_frame = False, None

try:
    while True:
        if cv2.getWindowProperty(WIN_PREV, cv2.WND_PROP_VISIBLE) < 1: break

        if not paused:
            ok, frame = cap.read()
            if not ok:
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0); continue
            if DOWNSCALE != 1.0:
                frame = cv2.resize(frame, None, fx=DOWNSCALE, fy=DOWNSCALE, interpolation=cv2.INTER_AREA)
            last_frame = frame
            adj, mask = build_mask(frame)
            cv2.imshow(WIN_PREV, make_mosaic(adj, mask))
        else:
            if last_frame is not None:
                adj, mask = build_mask(last_frame)
                cv2.imshow(WIN_PREV, make_mosaic(adj, mask))
            time.sleep(0.01)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'): break
        if key == ord(' '): paused = not paused
        if key == ord('s'):
            cs    = tb("ColorSpace 0=HSV 1=RGB", WIN_C1)
            alpha = tb("Alpha x100 (contrast)", WIN_C2) / 100.0
            beta  = tb("Beta (-100..100)+100",  WIN_C2) - 100
            fuse  = tb("Fuse Motion (0/1)",     WIN_C2)
            if cs == 0:
                color = {
                    "color_space": "HSV",
                    "H_low": tb("H low", WIN_C1),  "H_high": tb("H high", WIN_C1),
                    "S_low": tb("S low", WIN_C1),  "S_high": tb("S high", WIN_C1),
                    "V_low": tb("V low", WIN_C1),  "V_high": tb("V high", WIN_C1),
                }
            else:
                color = {
                    "color_space": "RGB",
                    "R_low": tb("R low", WIN_C1),  "R_high": tb("R high", WIN_C1),
                    "G_low": tb("G low", WIN_C1),  "G_high": tb("G high", WIN_C1),
                    "B_low": tb("B low", WIN_C1),  "B_high": tb("B high", WIN_C1),
                }
            cfg = {"color": color, "contrast_alpha": alpha, "brightness_beta": beta,
                   "fuse_motion": int(fuse), "downscale": DOWNSCALE}
            with open(SAVE_JSON, "w") as f:
                json.dump(cfg, f, indent=2)
            print(f"Saved → {SAVE_JSON.resolve()}")
finally:
    cap.release()
    cv2.destroyAllWindows()


Controls:  [S] Save JSON   [Space] Pause/Play   [Q] Quit


In [11]:
import cv2, json, csv, numpy as np
from pathlib import Path

SETTINGS_JSON = "outputs/mask_params.json"
VIDEO_PATH = "WhatsApp Video 2025-08-30 at 11.02.13 AM.mp4"

OUT = Path("outputs")
ANN_DIR  = OUT / "annotated"
MASK_DIR = OUT / "masks"
CROP_DIR = OUT / "crops"
for d in (ANN_DIR, MASK_DIR, CROP_DIR): d.mkdir(parents=True, exist_ok=True)

def load_settings(p=SETTINGS_JSON):
    with open(p, "r") as f:
        cfg = json.load(f)
    cfg.setdefault("downscale", 1.0)
    cfg.setdefault("contrast_alpha", 1.0)
    cfg.setdefault("brightness_beta", 0)
    cfg.setdefault("fuse_motion", 1)
    return cfg

def build_mask_from_settings(frame_bgr, cfg, bg_subtractor=None):
    alpha = float(cfg["contrast_alpha"])
    beta  = float(cfg["brightness_beta"])
    adj = cv2.convertScaleAbs(frame_bgr, alpha=alpha, beta=beta)

    if cfg["color"]["color_space"] == "HSV":
        H_low, H_high = cfg["color"]["H_low"], cfg["color"]["H_high"]
        S_low, S_high = cfg["color"]["S_low"], cfg["color"]["S_high"]
        V_low, V_high = cfg["color"]["V_low"], cfg["color"]["V_high"]
        hsv = cv2.cvtColor(adj, cv2.COLOR_BGR2HSV)
        if H_low <= H_high:
            lower = np.array([H_low, S_low, V_low], np.uint8)
            upper = np.array([H_high, S_high, V_high], np.uint8)
            mask_color = cv2.inRange(hsv, lower, upper)
        else:
            l1,u1 = np.array([0,  S_low, V_low], np.uint8), np.array([H_high, S_high, V_high], np.uint8)
            l2,u2 = np.array([H_low, S_low, V_low], np.uint8), np.array([179, S_high, V_high], np.uint8)
            mask_color = cv2.bitwise_or(cv2.inRange(hsv, l1, u1), cv2.inRange(hsv, l2, u2))
    else:
        rgb = cv2.cvtColor(adj, cv2.COLOR_BGR2RGB)
        R_low, R_high = cfg["color"]["R_low"], cfg["color"]["R_high"]
        G_low, G_high = cfg["color"]["G_low"], cfg["color"]["G_high"]
        B_low, B_high = cfg["color"]["B_low"], cfg["color"]["B_high"]
        lower = np.array([R_low, G_low, B_low], np.uint8)
        upper = np.array([R_high, G_high, B_high], np.uint8)
        mask_color = cv2.inRange(rgb, lower, upper)

    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    mask_color = cv2.morphologyEx(mask_color, cv2.MORPH_OPEN, k, iterations=1)
    mask_color = cv2.morphologyEx(mask_color, cv2.MORPH_CLOSE, k, iterations=1)

    if int(cfg.get("fuse_motion", 1)) == 1 and bg_subtractor is not None:
        fg = bg_subtractor.apply(adj)
        fg = cv2.medianBlur(fg, 5)
        mask = cv2.bitwise_and(mask_color, fg)
    else:
        mask = mask_color

    return adj, mask


In [12]:
from collections import deque

def process_video_with_mask(settings_path=SETTINGS_JSON,
                            video_path=VIDEO_PATH,
                            save_video=True,
                            draw_path=True,
                            trail_maxlen=10000,
                            # prediction params:
                            history_len=5,          # use last K measured points
                            enforce_horizontal=True,# keep y flat at median of K
                            dx_clip_px=50,          # cap horizontal step per miss
                            max_predict_frames=45): # max coasting length
    cfg = load_settings(settings_path)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError("Could not open video")

    DS = float(cfg.get("downscale", 1.0))
    bg = cv2.createBackgroundSubtractorMOG2(history=600, varThreshold=32, detectShadows=False) \
         if int(cfg.get("fuse_motion", 1)) == 1 else None

    OUT.mkdir(exist_ok=True, parents=True)
    csv_path = OUT / "detections.csv"
    with open(csv_path, "w", newline="") as f:
        csv.writer(f).writerow(["frame","x","y","w","h","source"])  # measured|predicted

    writer = None
    trail = deque(maxlen=trail_maxlen)
    measured_hist = deque(maxlen=history_len)
    last_pt = None
    miss_streak = 0

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret: break
        frame_idx += 1

        frame_ds = cv2.resize(frame, None, fx=DS, fy=DS, interpolation=cv2.INTER_AREA) if DS != 1.0 else frame
        adj, mask = build_mask_from_settings(frame_ds, cfg, bg)

        cnts = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts)==2 else cnts[1]
        H, W = adj.shape[:2]

        # pick best contour
        best = None
        best_score = -1.0
        for c in cnts:
            area = cv2.contourArea(c)
            if area < 8: continue
            M = cv2.moments(c)
            if M["m00"] == 0: continue
            cx, cy = int(M["m10"]/M["m00"]), int(M["m01"]/M["m00"])
            x,y,w,h = cv2.boundingRect(c)
            p = cv2.arcLength(c, True)
            circ = (4*np.pi*area/(p*p)) if p>0 else 0
            score = area*0.15 + circ*20
            if score > best_score:
                best_score = score
                best = (cx, cy, x, y, w, h)

        vis = adj.copy()

        if best is not None:
            cx, cy, x, y, w, h = best
            last_pt = (cx, cy)
            if draw_path:
                trail.append(last_pt)
            measured_hist.append((cx, cy))
            miss_streak = 0

            cv2.rectangle(vis, (x,y), (x+w,y+h), (0,255,0), 2)
            cv2.circle(vis, (cx,cy), 4, (0,0,255), -1)

            scale_back = 1.0 / DS
            with open(csv_path, "a", newline="") as f:
                csv.writer(f).writerow([frame_idx,
                                        int(x*scale_back), int(y*scale_back),
                                        int(w*scale_back), int(h*scale_back),
                                        "measured"])
        else:
            # predict horizontally using last K measured points
            miss_streak += 1
            if last_pt is None or len(measured_hist) < 2:
                if last_pt is not None and draw_path:
                    trail.append(last_pt)
                with open(csv_path, "a", newline="") as f:
                    csv.writer(f).writerow([frame_idx, 0, 0, 0, 0, "predicted"])
            else:
                xs = [p[0] for p in measured_hist]
                ys = [p[1] for p in measured_hist]
                dxs = [xs[i+1]-xs[i] for i in range(len(xs)-1)]
                dys = [ys[i+1]-ys[i] for i in range(len(ys)-1)]

                med_dx = float(np.median(dxs)) if dxs else 0.0
                med_dy = float(np.median(dys)) if dys else 0.0

                if enforce_horizontal:
                    next_y = int(np.median(ys))  # lock to median height
                    step_x = int(np.clip(med_dx, -dx_clip_px, dx_clip_px))
                    if step_x == 0:
                        step_x = 1 if (dxs and dxs[-1] >= 0) else -1
                    next_x = int(np.clip(last_pt[0] + step_x, 0, W-1))
                else:
                    step_x = int(np.clip(med_dx, -dx_clip_px, dx_clip_px))
                    step_y = int(np.clip(med_dy*0.2, -10, 10))
                    next_x = int(np.clip(last_pt[0] + step_x, 0, W-1))
                    next_y = int(np.clip(last_pt[1] + step_y, 0, H-1))

                last_pt = (next_x, next_y)
                if draw_path:
                    trail.append(last_pt)

                cv2.circle(vis, last_pt, 4, (0,165,255), -1)  # orange dot for predicted

                with open(csv_path, "a", newline="") as f:
                    csv.writer(f).writerow([frame_idx, 0, 0, 0, 0, "predicted"])

                if miss_streak > max_predict_frames:
                    pass  # stop drifting forever

        # draw full trail each frame
        if draw_path and len(trail) > 1:
            for i in range(1, len(trail)):
                cv2.line(vis, trail[i-1], trail[i], (0,255,0), 2)

        cv2.putText(vis, f"Frame {frame_idx}", (10,26),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)

        # writer
        if save_video and writer is None:
            fourcc = cv2.VideoWriter_fourcc(*"mp4v")
            fps = cap.get(cv2.CAP_PROP_FPS) or 30
            writer = cv2.VideoWriter(str(OUT / "isolated_annotated.mp4"), fourcc, fps, (vis.shape[1], vis.shape[0]))
        if writer is not None:
            writer.write(vis)

        # preview (press 'q' to stop)
        cv2.imshow("Isolated (annotated)", vis)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    if writer is not None:
        writer.release()
    cv2.destroyAllWindows()

    print(f"Done.\n- Video: {OUT.resolve()}/isolated_annotated.mp4\n- CSV:   {OUT.resolve()}/detections.csv\n- Masks: outputs/masks/\n- Crops: outputs/crops/")

# Run it
process_video_with_mask()


Done.
- Video: C:\Users\1033249\OneDrive - Blue Yonder\documents\GitHub\deepLearning_experiment_badmintan\outputs/isolated_annotated.mp4
- CSV:   C:\Users\1033249\OneDrive - Blue Yonder\documents\GitHub\deepLearning_experiment_badmintan\outputs/detections.csv
- Masks: outputs/masks/
- Crops: outputs/crops/


## Advance Code

In [13]:
import cv2, numpy as np, json
from pathlib import Path
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk

# ---------- Config ----------
video_path = "WhatsApp Video 2025-08-30 at 11.02.13 AM.mp4"
OUT_DIR = Path("outputs"); OUT_DIR.mkdir(parents=True, exist_ok=True)
SAVE_JSON = OUT_DIR / "mask_params.json"

DOWNSCALE = 1.0          # keep 1.0 for tiny shuttle
PREVIEW_MAX_W = 1280     # preview content caps (window can be larger)
PREVIEW_MAX_H = 720

# ---------- OpenCV capture ----------
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise RuntimeError("Could not open video (check path).")

bg = cv2.createBackgroundSubtractorMOG2(history=600, varThreshold=32, detectShadows=False)

# ---------- Tk roots: two separate windows ----------
root = tk.Tk()
root.withdraw()  # hide implicit root

w_preview = tk.Toplevel()
w_preview.title("Shuttle Preview")
w_preview.geometry("900x520+80+60")
w_preview.minsize(480, 320)

w_controls = tk.Toplevel()
w_controls.title("Shuttle Controls")
w_controls.geometry("540x640+1000+60")
w_controls.minsize(520, 620)

# ---- running flag so the cell exits cleanly on close ----
running = True

# ----- PREVIEW WINDOW -----
preview_canvas = tk.Label(w_preview, bg="#111")
preview_canvas.pack(fill="both", expand=True)

# ----- CONTROLS WINDOW -----
controls_frame = ttk.Frame(w_controls, padding=8)
controls_frame.pack(fill="both", expand=True)

def make_labeled_scale(parent, text, var, frm, to, row, col=0, length=220, fmt="{:d}"):
    ttk.Label(parent, text=text).grid(row=row, column=col, sticky="w")
    s = ttk.Scale(parent, from_=frm, to=to, orient="horizontal",
                  variable=var, length=length)
    s.grid(row=row, column=col+1, padx=6, pady=3, sticky="ew")
    val = ttk.Label(parent, width=6)
    val.grid(row=row, column=col+2, sticky="w")
    def _upd(*_):
        v = var.get()
        try: val.config(text=fmt.format(int(round(v))))
        except Exception: val.config(text=fmt.format(float(v)))
    var.trace_add("write", _upd); _upd()
    return s

# groups
col_left  = ttk.LabelFrame(controls_frame, text="Color Space & Ranges", padding=8)
col_right = ttk.LabelFrame(controls_frame, text="Levels & Options", padding=8)
col_left.grid(row=0, column=0, sticky="n", padx=(0,8))
col_right.grid(row=0, column=1, sticky="n")

view_group = ttk.LabelFrame(controls_frame, text="Preview", padding=8)
view_group.grid(row=1, column=0, columnspan=2, sticky="ew", pady=(8,0))

# --- view & zoom ---
view_mode = tk.StringVar(value="Original")
for i, name in enumerate(("Original","Mask","Masked","Mosaic")):
    ttk.Radiobutton(view_group, text=name, variable=view_mode, value=name).grid(row=0, column=i, sticky="w")
zoom_var = tk.IntVar(value=100)
ttk.Label(view_group, text="Zoom %").grid(row=1, column=0, sticky="w", pady=(6,0))
make_labeled_scale(view_group, " ", zoom_var, 50, 200, row=1, col=1, length=260, fmt="{:d}")

# --- color space toggle ---
color_space = tk.StringVar(value="HSV")
ttk.Radiobutton(col_left, text="HSV", variable=color_space, value="HSV").grid(row=0, column=0, sticky="w")
ttk.Radiobutton(col_left, text="RGB", variable=color_space, value="RGB").grid(row=0, column=1, sticky="w")

# --- HSV sliders ---
H_low  = tk.IntVar(value=30);  H_high = tk.IntVar(value=60)
S_low  = tk.IntVar(value=120); S_high = tk.IntVar(value=255)
V_low  = tk.IntVar(value=150); V_high = tk.IntVar(value=255)
make_labeled_scale(col_left, "H low",  H_low,  0, 179, 1)
make_labeled_scale(col_left, "H high", H_high, 0, 179, 2)
make_labeled_scale(col_left, "S low",  S_low,  0, 255, 3)
make_labeled_scale(col_left, "S high", S_high, 0, 255, 4)
make_labeled_scale(col_left, "V low",  V_low,  0, 255, 5)
make_labeled_scale(col_left, "V high", V_high, 0, 255, 6)

# --- RGB sliders ---
R_low  = tk.IntVar(value=0);   R_high = tk.IntVar(value=255)
G_low  = tk.IntVar(value=180); G_high = tk.IntVar(value=255)
B_low  = tk.IntVar(value=0);   B_high = tk.IntVar(value=120)
make_labeled_scale(col_left, "R low",  R_low,  0, 255, 7)
make_labeled_scale(col_left, "R high", R_high, 0, 255, 8)
make_labeled_scale(col_left, "G low",  G_low,  0, 255, 9)
make_labeled_scale(col_left, "G high", G_high, 0, 255,10)
make_labeled_scale(col_left, "B low",  B_low,  0, 255,11)
make_labeled_scale(col_left, "B high", B_high, 0, 255,12)

# --- levels & options ---
alpha_var = tk.DoubleVar(value=1.00)
beta_var  = tk.IntVar(value=0)
fuse_motion = tk.IntVar(value=1)

make_labeled_scale(col_right, "Contrast (alpha)", alpha_var, 0.5, 4.0, 0, fmt="{:.2f}")
make_labeled_scale(col_right, "Brightness (beta)", beta_var, -100, 100, 1, fmt="{:d}")
ttk.Checkbutton(col_right, text="Fuse Motion (MOG2)", variable=fuse_motion).grid(row=2, column=0, columnspan=3, sticky="w", pady=(6,2))

def save_settings():
    if color_space.get() == "HSV":
        color = {"color_space":"HSV",
                 "H_low":H_low.get(), "H_high":H_high.get(),
                 "S_low":S_low.get(), "S_high":S_high.get(),
                 "V_low":V_low.get(), "V_high":V_high.get()}
    else:
        color = {"color_space":"RGB",
                 "R_low":R_low.get(), "R_high":R_high.get(),
                 "G_low":G_low.get(), "G_high":G_high.get(),
                 "B_low":B_low.get(), "B_high":B_high.get()}
    cfg = {"color":color,
           "contrast_alpha":float(alpha_var.get()),
           "brightness_beta":int(beta_var.get()),
           "fuse_motion":int(fuse_motion.get()),
           "downscale":DOWNSCALE}
    with open(SAVE_JSON, "w") as f:
        json.dump(cfg, f, indent=2)
    status_var.set(f"Saved → {SAVE_JSON.resolve()}")

ttk.Button(col_right, text="Save Settings", command=save_settings).grid(row=3, column=0, columnspan=3, pady=10, sticky="ew")
status_var = tk.StringVar(value="Pick a view, tune sliders, then Save Settings")
ttk.Label(col_right, textvariable=status_var, wraplength=320, foreground="#0a6").grid(row=4, column=0, columnspan=3, sticky="w")

# ---------- Mask + preview helpers ----------
def build_mask(frame_bgr):
    alpha = float(alpha_var.get())
    beta  = int(beta_var.get())
    adj = cv2.convertScaleAbs(frame_bgr, alpha=alpha, beta=beta)

    if color_space.get() == "HSV":
        hL,hH = int(H_low.get()),  int(H_high.get())
        sL,sH = int(S_low.get()),  int(S_high.get())
        vL,vH = int(V_low.get()),  int(V_high.get())
        hsv = cv2.cvtColor(adj, cv2.COLOR_BGR2HSV)
        if hL <= hH:
            lower = np.array([hL, sL, vL], np.uint8)
            upper = np.array([hH, sH, vH], np.uint8)
            mask_color = cv2.inRange(hsv, lower, upper)
        else:
            l1,u1 = np.array([0,  sL, vL], np.uint8), np.array([hH, sH, vH], np.uint8)
            l2,u2 = np.array([hL, sL, vL], np.uint8), np.array([179, sH, vH], np.uint8)
            mask_color = cv2.bitwise_or(cv2.inRange(hsv, l1, u1), cv2.inRange(hsv, l2, u2))
    else:
        rgb = cv2.cvtColor(adj, cv2.COLOR_BGR2RGB)
        lower = np.array([R_low.get(), G_low.get(), B_low.get()], np.uint8)
        upper = np.array([R_high.get(), G_high.get(), B_high.get()], np.uint8)
        mask_color = cv2.inRange(rgb, lower, upper)

    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    mask_color = cv2.morphologyEx(mask_color, cv2.MORPH_OPEN, k, iterations=1)
    mask_color = cv2.morphologyEx(mask_color, cv2.MORPH_CLOSE, k, iterations=1)

    if fuse_motion.get() == 1:
        fg = bg.apply(adj)
        fg = cv2.medianBlur(fg, 5)
        mask = cv2.bitwise_and(mask_color, fg)
    else:
        mask = mask_color
    return adj, mask

def fit_scale(w, h, max_w, max_h, zoom_pct):
    z = max(0.01, zoom_pct/100.0)
    w2, h2 = int(w*z), int(h*z)
    s = min(max_w/max(w2,1), max_h/max(h2,1), 1.0)
    return z*s

def render_view(adj, mask, mode):
    if mode == "Original":
        view = adj
    elif mode == "Mask":
        view = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    elif mode == "Masked":
        view = cv2.bitwise_and(adj, adj, mask=mask)
    else:  # Mosaic
        mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        masked   = cv2.bitwise_and(adj, adj, mask=mask)
        view = np.hstack([adj, mask_bgr, masked])
    return view

def update_preview():
    if not running:
        return  # stop scheduling when closing

    w_preview.update_idletasks()
    max_w = max(200, w_preview.winfo_width()-16)
    max_h = max(150, w_preview.winfo_height()-16)

    ok, frame = cap.read()
    if not ok or not running:
        if running:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            w_preview.after(10, update_preview)
        return

    if DOWNSCALE != 1.0:
        frame = cv2.resize(frame, None, fx=DOWNSCALE, fy=DOWNSCALE, interpolation=cv2.INTER_AREA)

    adj, mask = build_mask(frame)
    view = render_view(adj, mask, view_mode.get())

    # cap size by both the window and global caps
    max_w = min(max_w, PREVIEW_MAX_W)
    max_h = min(max_h, PREVIEW_MAX_H)
    h, w = view.shape[:2]
    scale = fit_scale(w, h, max_w, max_h, zoom_var.get())
    if abs(scale-1.0) > 1e-6:
        view = cv2.resize(view, (int(w*scale), int(h*scale)), interpolation=cv2.INTER_AREA)

    view_rgb = cv2.cvtColor(view, cv2.COLOR_BGR2RGB)
    im = Image.fromarray(view_rgb)
    tk_img = ImageTk.PhotoImage(image=im)
    preview_canvas.configure(image=tk_img)
    preview_canvas.image = tk_img

    if running:
        w_preview.after(1, update_preview)

def on_close_all():
    global running
    running = False
    try: cap.release()
    except Exception: pass
    for w in (w_preview, w_controls):
        try: w.destroy()
        except Exception: pass
    try:
        root.quit()      # stop mainloop
        root.destroy()   # kill hidden root
    except Exception:
        pass

w_preview.protocol("WM_DELETE_WINDOW", on_close_all)
w_controls.protocol("WM_DELETE_WINDOW", on_close_all)

update_preview()
tk.mainloop()


In [16]:
# CELL 1 — helpers
import cv2, json, csv, numpy as np
from pathlib import Path

SETTINGS_JSON = "outputs/mask_params.json"
VIDEO_PATH    = "WhatsApp Video 2025-08-30 at 11.02.13 AM.mp4"

# outputs
OUT      = Path("outputs"); OUT.mkdir(parents=True, exist_ok=True)
MASK_DIR = OUT / "masks";   MASK_DIR.mkdir(exist_ok=True)
CROP_DIR = OUT / "crops";   CROP_DIR.mkdir(exist_ok=True)

def load_settings(p=SETTINGS_JSON):
    with open(p, "r") as f:
        cfg = json.load(f)
    cfg.setdefault("downscale", 1.0)
    cfg.setdefault("contrast_alpha", 1.0)
    cfg.setdefault("brightness_beta", 0)
    cfg.setdefault("fuse_motion", 1)
    return cfg

def build_mask_from_settings(frame_bgr, cfg, bg_subtractor=None):
    """Return (adjusted_frame, binary_mask) using your saved thresholds."""
    alpha = float(cfg["contrast_alpha"])
    beta  = float(cfg["brightness_beta"])
    adj = cv2.convertScaleAbs(frame_bgr, alpha=alpha, beta=beta)

    if cfg["color"]["color_space"] == "HSV":
        H_low, H_high = cfg["color"]["H_low"],  cfg["color"]["H_high"]
        S_low, S_high = cfg["color"]["S_low"],  cfg["color"]["S_high"]
        V_low, V_high = cfg["color"]["V_low"],  cfg["color"]["V_high"]
        hsv = cv2.cvtColor(adj, cv2.COLOR_BGR2HSV)
        if H_low <= H_high:
            lower = np.array([H_low, S_low, V_low], np.uint8)
            upper = np.array([H_high, S_high, V_high], np.uint8)
            mask_color = cv2.inRange(hsv, lower, upper)
        else:
            l1,u1 = np.array([0,   S_low, V_low], np.uint8), np.array([H_high, S_high, V_high], np.uint8)
            l2,u2 = np.array([H_low, S_low, V_low], np.uint8), np.array([179,  S_high, V_high], np.uint8)
            mask_color = cv2.bitwise_or(cv2.inRange(hsv, l1, u1), cv2.inRange(hsv, l2, u2))
    else:
        rgb = cv2.cvtColor(adj, cv2.COLOR_BGR2RGB)
        R_low, R_high = cfg["color"]["R_low"], cfg["color"]["R_high"]
        G_low, G_high = cfg["color"]["G_low"], cfg["color"]["G_high"]
        B_low, B_high = cfg["color"]["B_low"], cfg["color"]["B_high"]
        lower = np.array([R_low, G_low, B_low], np.uint8)
        upper = np.array([R_high, G_high, B_high], np.uint8)
        mask_color = cv2.inRange(rgb, lower, upper)

    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    mask_color = cv2.morphologyEx(mask_color, cv2.MORPH_OPEN, k, iterations=1)
    mask_color = cv2.morphologyEx(mask_color, cv2.MORPH_CLOSE, k, iterations=1)

    if int(cfg.get("fuse_motion", 1)) == 1 and bg_subtractor is not None:
        fg = bg_subtractor.apply(adj)
        fg = cv2.medianBlur(fg, 5)
        mask = cv2.bitwise_and(mask_color, fg)
    else:
        mask = mask_color

    return adj, mask


In [20]:
# CELL 2 — robust left/right during misses: keep last measured velocity
from collections import deque
import numpy as np, csv, cv2

def _estimate_fps(video_path, fallback=30.0):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened(): return fallback
    fps = cap.get(cv2.CAP_PROP_FPS) or 0
    cap.release()
    return fps if 1.0 <= fps <= 240.0 else fallback

def process_video_with_mask(settings_path=SETTINGS_JSON,
                            video_path=VIDEO_PATH,
                            save_video=True,
                            draw_path=True,
                            trail_maxlen=20000,
                            # velocity estimation window (measured-only):
                            vel_k=5,                  # use last K measured frames for velocity
                            vel_decay=0.98,           # decay when coasting (1.0 = no decay)
                            enforce_horizontal=True,  # lock Y during misses
                            max_step_px=80,           # clamp |Δx| per frame in DS px
                            min_step_px=1,            # min |Δx| when coasting
                            max_predict_frames=90):   # don’t coast forever
    cfg = load_settings(settings_path)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError("Could not open video")

    DS = float(cfg.get("downscale", 1.0)) or 1.0
    scale_back = 1.0 / DS
    fps_out = _estimate_fps(video_path)

    # enable motion fusion only for mask-making
    bg = cv2.createBackgroundSubtractorMOG2(history=600, varThreshold=32, detectShadows=False) \
         if int(cfg.get("fuse_motion", 1)) == 1 else None

    # outputs
    OUT.mkdir(exist_ok=True, parents=True)
    csv_path = OUT / "detections.csv"
    with open(csv_path, "w", newline="") as f:
        csv.writer(f).writerow(["frame","x","y","w","h","source"])  # ORIGINAL coords

    writer = None

    # trail drawn on ORIGINAL coords
    trail_orig = deque(maxlen=trail_maxlen)

    # keep only MEASURED centers (DS coords) + their absolute frame indices for velocity fit
    meas_pts_ds = deque(maxlen=max(vel_k, 6))     # [(frame_idx, cx_ds, cy_ds), ...]
    last_used_ds = None                           # last point in DS we advanced from (measured or predicted)
    vx_ds = 0.0                                   # current horizontal velocity (DS px/frame), signed
    miss_streak = 0
    frame_idx = 0

    while True:
        ret, frame_orig = cap.read()
        if not ret:
            break
        frame_idx += 1

        H0, W0 = frame_orig.shape[:2]
        frame_ds = cv2.resize(frame_orig, None, fx=DS, fy=DS, interpolation=cv2.INTER_AREA) if DS != 1.0 else frame_orig

        adj_ds, mask = build_mask_from_settings(frame_ds, cfg, bg)
        cnts = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts)==2 else cnts[1]

        # choose best contour (small + roundish)
        best = None; best_score = -1.0
        for c in cnts:
            area = cv2.contourArea(c)
            if area < 8: continue
            M = cv2.moments(c)
            if M["m00"] == 0: continue
            cx, cy = int(M["m10"]/M["m00"]), int(M["m01"]/M["m00"])
            x,y,w,h = cv2.boundingRect(c)
            p = cv2.arcLength(c, True)
            circ = (4*np.pi*area/(p*p)) if p>0 else 0
            score = area*0.15 + circ*20
            if score > best_score:
                best_score = score
                best = (cx, cy, x, y, w, h)

        vis = frame_orig.copy()

        if best is not None:
            # ------ MEASURED ------
            cx, cy, x, y, w, h = best
            miss_streak = 0
            last_used_ds = (cx, cy)

            # append measured point for velocity fit
            meas_pts_ds.append((frame_idx, cx, cy))

            # update velocity using last vel_k measured points
            if len(meas_pts_ds) >= 2:
                recent = list(meas_pts_ds)[-vel_k:]
                t = np.array([p[0] for p in recent], dtype=np.float32)
                xarr = np.array([p[1] for p in recent], dtype=np.float32)
                # robust small window linear fit: x ≈ a*t + b -> a is vx
                # center times to reduce numeric issues
                t0 = t.mean()
                denom = np.sum((t - t0)**2)
                if denom > 0:
                    a = np.sum((t - t0) * (xarr - xarr.mean())) / denom
                    vx_ds = float(np.clip(a, -max_step_px, max_step_px))
                # if velocity is tiny but the last step was not, keep last vx_ds
                if abs(vx_ds) < 0.25 and len(recent) >= 2:
                    dx_last = recent[-1][1] - recent[-2][1]
                    if abs(dx_last) >= 1:
                        vx_ds = float(np.clip(dx_last, -max_step_px, max_step_px))

            # map to ORIGINAL and draw
            X = int(round(x * scale_back)); Y = int(round(y * scale_back))
            Wb = int(round(w * scale_back)); Hb = int(round(h * scale_back))
            Cx = int(round(cx * scale_back)); Cy = int(round(cy * scale_back))

            if draw_path:
                trail_orig.append((Cx, Cy))
            cv2.rectangle(vis, (X, Y), (X+Wb, Y+Hb), (0,255,0), 2)
            cv2.circle(vis, (Cx, Cy), 4, (0,0,255), -1)

            with open(csv_path, "a", newline="") as f:
                csv.writer(f).writerow([frame_idx, X, Y, Wb, Hb, "measured"])

        else:
            # ------ PREDICT (use last measured velocity strictly) ------
            miss_streak += 1
            if last_used_ds is None or len(meas_pts_ds) < 2:
                # hold trail steady until we have velocity
                if len(trail_orig) > 0 and draw_path:
                    trail_orig.append(trail_orig[-1])
                with open(csv_path, "a", newline="") as f:
                    csv.writer(f).writerow([frame_idx, 0, 0, 0, 0, "predicted"])
            else:
                # speed = last vx_ds (signed). If zero, fall back to last measured step.
                if abs(vx_ds) < 0.25:
                    # try last instantaneous step from the last two measured points
                    p2, p1 = meas_pts_ds[-1], meas_pts_ds[-2]
                    vx_ds = float(np.clip(p2[1] - p1[1], -max_step_px, max_step_px))
                    if abs(vx_ds) < 0.25:
                        vx_ds = min_step_px  # final fallback to small positive step
                # clamp & apply decay while coasting so it doesn't accelerate
                step_x_ds = float(np.clip(vx_ds, -max_step_px, max_step_px))
                vx_ds *= float(vel_decay)

                # anchor: advance from last_used_ds (the last measured/predicted point)
                base_x_ds, base_y_ds = last_used_ds
                # lock Y to recent median (measured-only)
                ys_recent = [p[2] for p in list(meas_pts_ds)[-vel_k:]]
                next_y_ds = int(np.median(ys_recent)) if enforce_horizontal else base_y_ds

                Hds, Wds = adj_ds.shape[:2]
                next_x_ds = int(np.clip(base_x_ds + step_x_ds, 0, Wds-1))
                last_used_ds = (next_x_ds, next_y_ds)

                # map to ORIGINAL
                Cx = int(round(next_x_ds * scale_back))
                Cy = int(round(next_y_ds * scale_back))
                if draw_path:
                    trail_orig.append((Cx, Cy))
                cv2.circle(vis, (Cx, Cy), 4, (0,165,255), -1)

                with open(csv_path, "a", newline="") as f:
                    csv.writer(f).writerow([frame_idx, 0, 0, 0, 0, "predicted"])

                if miss_streak > max_predict_frames:
                    pass

        # draw the full trail on ORIGINAL
        if draw_path and len(trail_orig) > 1:
            for i in range(1, len(trail_orig)):
                cv2.line(vis, trail_orig[i-1], trail_orig[i], (0,255,0), 2)

        cv2.putText(vis, f"Frame {frame_idx}", (10,26),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)

        # writer (ORIGINAL size, source FPS)
        if save_video and writer is None:
            fourcc = cv2.VideoWriter_fourcc(*"mp4v")
            writer = cv2.VideoWriter(str(OUT / "isolated_annotated.mp4"), fourcc, fps_out, (W0, H0))
        if writer is not None:
            writer.write(vis)

        # optional live preview
        cv2.imshow("Isolated (annotated, ORIGINAL size)", vis)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    if writer is not None:
        writer.release()
    cv2.destroyAllWindows()

    print(f"Done.\n- Video (original size): {OUT.resolve()}/isolated_annotated.mp4 @ {fps_out:.2f} FPS\n- CSV: {OUT.resolve()}/detections.csv")

# Run it
process_video_with_mask()


Done.
- Video (original size): C:\Users\1033249\OneDrive - Blue Yonder\documents\GitHub\deepLearning_experiment_badmintan\outputs/isolated_annotated.mp4 @ 9.49 FPS
- CSV: C:\Users\1033249\OneDrive - Blue Yonder\documents\GitHub\deepLearning_experiment_badmintan\outputs/detections.csv
